In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import sklearn.metrics as sm
from sklearn.model_selection import cross_validate
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import GradientBoostingClassifier as grad
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.tree import DecisionTreeClassifier as tree
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
import scipy.stats as stats
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, "../src/")
import util as util

%autosave 5

In [ ]:
datingTrain = pd.read_csv('../data/plotlyDashData/datingTrain.csv')
match = datingTrain["match"]
X = datingTrain.drop("match",axis=1).select_dtypes(include=['uint8','int64','float64'])
with open('../data/plotlyDashData/treeParams.json') as d:
    treeParams = json.load(d)
    preciseTreeParams = treeParams["preciseTreeParams"]
    recallTreeParams = treeParams["recallTreeParams"]
with open('../data/plotlyDashData/forestParams.json') as d:
    forestParams = json.load(d)
    preciseForestParams = forestParams["preciseForestParams"]
    recallForestParams = forestParams["recallForestParams"]

In [ ]:
sqrtn = int(np.sqrt(X.shape[0]))
logModel = lm.LogisticRegression(max_iter=1e9)
logPipe = make_pipeline(StandardScaler(), logModel)
knn5 = knn(n_neighbors=5)
knnsqrtn = knn(n_neighbors=sqrtn)
gradientdeci = grad(learning_rate=0.1)
gradientdeka = grad(learning_rate=10)
preciseTree = tree(criterion = preciseTreeParams["criterion"],
                    max_depth = preciseTreeParams["max_depth"],
                    max_features = preciseTreeParams["max_features"])
recallTree = tree(criterion = recallTreeParams["criterion"],
                  max_depth = recallTreeParams["max_depth"],
                  max_features = recallTreeParams["max_features"])
preciseForest = rf(n_estimators = preciseForestParams["n_estimators"],
                    criterion = preciseForestParams["criterion"],
                    max_depth = preciseForestParams["max_depth"],
                    max_features = preciseForestParams["max_features"])
recallForest = rf(n_estimators = recallForestParams["n_estimators"],
                  criterion = recallForestParams["criterion"],
                  max_depth = recallForestParams["max_depth"],
                  max_features = recallForestParams["max_features"])

ensembleVote = VotingClassifier(
    estimators = [
        ("logModel",logPipe),
        ("knn5",knn5),
        ("knnsqrtn",knnsqrtn),
        ("gradientdeci",gradientdeci),
        ("gradientdeka",gradientdeka),
        ("preciseTree",preciseTree),
        ("recallTree",recallTree),
        ("preciseForest",preciseForest),
        ("recallForest",recallForest)
    ], voting="soft"
)


In [ ]:
import time
values = [0] * 10
for i in range(10):
    start_time = time.time()
    ensembleVote.fit(X,match)
    values[i] = time.time() - start_time
print(f"{np.mean(values)} +/- {np.mean(values)/np.sqrt(10)} seconds")

In [ ]:
values = [0] * 10
for i in range(10):
    start_time = time.time()
    np.array(ensembleVote.predict_proba(X)[:,1]).reshape(-1,).tolist()
    values[i] = time.time() - start_time
print(f"{np.mean(values)} +/- {np.mean(values)/np.sqrt(10)} seconds")

In [ ]:
values = [0] * 10
for i in range(10):
    start_time = time.time()
    ensembleVote.fit(X,match)
    np.array(ensembleVote.predict_proba(X)[:,1]).reshape(-1,).tolist()
    values[i] = time.time() - start_time
print(f"{np.mean(values)} +/- {np.mean(values)/np.sqrt(10)} seconds")